In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
torch.cuda.is_available()

True

In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.models import RNNModel, ComplexRNNModel
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import evaluate, training_loop

# MASS TRAINING (Humans achieve 46% +- 29%)

## Read dataset and preprocess it

In [4]:
def transform_velocity_to_speed_and_angle(data):
    OBJECTS = ["o1", "o2", "o3", "o4"]
    
    for i, obj in enumerate(OBJECTS):
        data[obj+".vx"] = np.sqrt(data[obj+".vx"]**2 + data[obj+".vy"]**2)        
        data[obj+".vy"] = np.arctan(data[obj+".vy"] / data[obj+".vx"])
        data[obj+".vy"].fillna(0, inplace=True)
        
def add_distance_between_objects(data):
    OBJECTS = ["o1", "o2", "o3", "o4"]

    for i, obj in enumerate(OBJECTS):
        for obj_two in OBJECTS[i+1:]: 
            data["_".join(["d", obj, obj_two])] = np.sqrt((data[obj+".x"] - data[obj_two+".x"])**2 +
                                                            (data[obj+".y"] - data[obj_two+".y"])**2)

In [5]:
all_trials = read_dataset("data/passive_trials.h5")
train_loader, val_loader, scaler = prepare_dataset(all_trials, MASS_CLASS_COLS, normalise_data=True, 
                                                   batch_size=64, test_size=0.3, equiprobable_training_classes=False,
                                                   sliding_window_size=1)

100%|██████████| 1000/1000 [00:00<00:00, 1110.88it/s]


In [6]:
Y = []
for x,y in train_loader:
    
    Y.extend(list(y))

counts = np.unique(Y, return_counts=True)[1]

In [7]:
counts

array([233, 222, 245])

In [8]:
print("Majority class: ", np.max(counts) / np.sum(counts))

Majority class:  0.35


In [9]:
for x, y in train_loader:
    print(x.shape)
    break

torch.Size([64, 2701, 16])


## Define model, loss and optimizer

In [15]:
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

input_dim = 16  # input dimension
hidden_dim = 32  # hidden layer dimension
n_layers = 2     # number of hidden layers
output_dim = 3   # output dimension

model = ComplexRNNModel(input_dim, hidden_dim, n_layers, output_dim, dropout=0.8)
model = model.cuda()

error = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
model.train()

ComplexRNNModel(
  (lstm): GRU(16, 32, num_layers=2, batch_first=True, dropout=0.8)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)

## Train model and plot loss and accuracy

In [17]:
epoch_losses, epoch_accuracies, _ = training_loop(model, optimizer, error, train_loader, val_loader, num_epochs=500)

Train_loss (0.04)	 Train_acc (84.43)	 Val_acc (46.33): 100%|██████████| 500/500 [19:26<00:00,  2.36s/it]


ValueError: too many values to unpack (expected 2)

In [18]:
plt.plot(epoch_losses)
plt.show()
plt.plot(np.array(epoch_accuracies).T)
plt.show()

NameError: name 'epoch_losses' is not defined

In [ ]:
max(epoch_accuracies[1])

## Save model and scaler

In [ ]:
torch.save(model.state_dict(), "models/passive_mass_model.pt")

In [ ]:
joblib.dump(scaler, "scalers/passive_mass_scaler.sk")

## Load model and evaluate

In [ ]:
model = ComplexRNNModel(input_dim, first_hidden_dim, second_hidden_dim, output_dim)
model.load_state_dict(torch.load("models/passive_mass_model.pt"))
model.eval()
model = model.cuda()

In [ ]:
accuracy, predicted = evaluate(model, val_loader, return_predicted=True)

In [ ]:
print(accuracy)

In [ ]:
predicted = [pred.cpu() for pred in predicted]
Y_val = np.concatenate([y.cpu().numpy() for x, y in val_loader])

In [ ]:
plot_confusion_matrix(Y_val, predicted, classes=MASS_CLASS_COLS, normalize=False)
plot_confusion_matrix(Y_val, predicted, classes=MASS_CLASS_COLS, normalize=True)

# FORCE TRAINING  (Humans achieve 61% +- 22%)

## Read dataset and preprocess it

In [ ]:
all_trials = read_dataset("data/passive_trials.h5")
train_loader, val_loader, scaler = prepare_dataset(all_trials, FORCE_CLASS_COLS, 
                                                   normalise_data=True, batch_size=128, test_size=0.3)

In [ ]:
Y = []
for x,y in train_loader:
    
    Y.extend(list(y))

counts = np.unique(Y, return_counts=True)[1]

In [ ]:
print("Majority class: ", np.max(counts) / np.sum(counts))

## Define model, loss and optimizer

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

input_dim = len(BASIC_TRAINING_COLS)    # input dimension
first_hidden_dim = 12  # hidden layer dimension
second_hidden_dim = 6     # number of hidden layers
output_dim = 3   # output dimension

model = ComplexRNNModel(input_dim, first_hidden_dim, second_hidden_dim, output_dim)
model = model.cuda()

error = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Train model and plot loss and accuracy

In [ ]:
epoch_losses, epoch_accuracies = training_loop(model, optimizer, error, train_loader, val_loader, num_epochs=1000)

In [ ]:
plt.plot(epoch_losses)
plt.show()
plt.plot(np.array(epoch_accuracies).T)
plt.show()

In [ ]:
max(epoch_accuracies[1])

## Save model and scaler

In [ ]:
torch.save(model.state_dict(), "models/passive_force_model.pt")

In [ ]:
joblib.dump(scaler, "scalers/passive_force_scaler.sk")

## Load model and evaluate

In [ ]:
model = ComplexRNNModel(input_dim, first_hidden_dim, second_hidden_dim, output_dim)
model.load_state_dict(torch.load("models/passive_force_model.pt"))
model.eval()
model = model.cuda()

In [ ]:
accuracy, predicted = evaluate(model, val_loader, return_predicted=True)

In [ ]:
print(accuracy)

In [ ]:
predicted = [pred.cpu() for pred in predicted]
Y_val = np.concatenate([y.cpu().numpy() for x, y in val_loader])

In [ ]:
plot_confusion_matrix(Y_val, predicted, classes=FORCE_CLASS_COLS, normalize=False)
plot_confusion_matrix(Y_val, predicted, classes=FORCE_CLASS_COLS, normalize=True)